In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, make_scorer
from sklearn.decomposition import PCA

In [2]:
path_archivo = "../input_data/combi_500_100.csv"
datos_combi  = pd.read_csv(path_archivo)

In [3]:
datos_combi = datos_combi.sort_values(by=["eta", "beta", "phi_K"]).reset_index(drop=True)

In [4]:
dat_copy = datos_combi.copy()
operacion = lambda x: np.log(x)
for i in range(10):
    key_mod = "eig_" + str(i+1)
    prev_key = "eig_" + str(i)
    if i == 0:  
        dat_copy[key_mod] = 1/dat_copy[key_mod]
    else:
        dat_copy[key_mod] = dat_copy[prev_key]/dat_copy[key_mod]
    #fin if 
    #dat_copy[key_mod] = np.array(list(map(operacion, dat_copy[key_mod])))
#fin for
#dat_copy["mu_sobre_K"] = np.tan(dat_copy["phi_K"])
datos_combi = dat_copy.copy()


In [5]:
datos_combi.head()

,phi_K,eta,beta,x_K,x_mu,eig_0,eig_1,eig_2,eig_3,eig_4,...,eig_90,eig_91,eig_92,eig_93,eig_94,eig_95,eig_96,eig_97,eig_98,eig_99
0,0.002000,0.19635,1.047198,0.999998,0.002000,3.910766e-07,0.123874,0.009468,0.000346,0.00001,...,427443.935227,446916.371010,448827.125228,449977.613611,458726.245203,458940.189236,521446.826866,525588.873740,583373.344465,583607.080825
1,0.005138,0.19635,1.047198,0.999987,0.005138,1.003534e-06,0.123881,0.009468,0.000346,0.00001,...,427718.567842,447296.943246,449205.482004,450245.199365,459202.763203,459406.655524,521990.068226,526127.246576,583326.887609,583563.229874
2,0.008275,0.19635,1.047198,0.999966,0.008275,1.614694e-06,0.123887,0.009468,0.000345,0.00001,...,427993.234411,447677.888893,449584.234077,450511.485953,459679.284087,459873.134636,522533.310087,526665.556822,583279.715703,583518.646462
3,0.011413,0.19635,1.047198,0.999935,0.011413,2.224556e-06,0.123893,0.009468,0.000345,0.00001,...,428267.942931,448059.213137,449963.386266,450776.472028,460155.819486,460339.637726,523076.564859,527203.814226,583231.833940,583473.335731
4,0.014550,0.19635,1.047198,0.999894,0.014550,2.833116e-06,0.123899,0.009468,0.000344,0.00001,...,428542.698070,448440.917760,450342.939976,451040.152614,460632.377451,460806.172377,523619.840868,527742.024324,583183.242886,583427.298198


In [6]:
N_eig = 20
for i in range(N_eig):
    col = "eig_" + str(i+1)
    datos_combi[col] = np.log(datos_combi[col])
#fin for 
datos_combi = datos_combi.copy()
datos_combi.head()

,phi_K,eta,beta,x_K,x_mu,eig_0,eig_1,eig_2,eig_3,eig_4,...,eig_90,eig_91,eig_92,eig_93,eig_94,eig_95,eig_96,eig_97,eig_98,eig_99
0,0.002000,0.19635,1.047198,0.999998,0.002000,3.910766e-07,-2.088487,-4.659885,-7.969479,-11.478219,...,427443.935227,446916.371010,448827.125228,449977.613611,458726.245203,458940.189236,521446.826866,525588.873740,583373.344465,583607.080825
1,0.005138,0.19635,1.047198,0.999987,0.005138,1.003534e-06,-2.088436,-4.659868,-7.970508,-11.479121,...,427718.567842,447296.943246,449205.482004,450245.199365,459202.763203,459406.655524,521990.068226,526127.246576,583326.887609,583563.229874
2,0.008275,0.19635,1.047198,0.999966,0.008275,1.614694e-06,-2.088385,-4.659852,-7.971537,-11.480022,...,427993.234411,447677.888893,449584.234077,450511.485953,459679.284087,459873.134636,522533.310087,526665.556822,583279.715703,583518.646462
3,0.011413,0.19635,1.047198,0.999935,0.011413,2.224556e-06,-2.088335,-4.659835,-7.972564,-11.480924,...,428267.942931,448059.213137,449963.386266,450776.472028,460155.819486,460339.637726,523076.564859,527203.814226,583231.833940,583473.335731
4,0.014550,0.19635,1.047198,0.999894,0.014550,2.833116e-06,-2.088285,-4.659819,-7.973591,-11.481824,...,428542.698070,448440.917760,450342.939976,451040.152614,460632.377451,460806.172377,523619.840868,527742.024324,583183.242886,583427.298198


In [7]:
N_phi_K = 500
N_datos = len(datos_combi)
N_partes = int(N_datos/N_phi_K)
print(N_partes)

105


In [8]:
def graficar_lambdas(data_frame, target, n_eig, preds = {}, op = lambda x: x, nombre = "lineal/", nombre_feat = "lambda_moño"):
    cols_eig = list(map(lambda x: "pca" + str(x), range(n_eig)))
    cols_preds = list(map(lambda x: "pred_" + x, cols_eig)) if len(preds) != 0 else []
    eta = str(np.array(data_frame["eta"])[0]/np.pi)[:4] + "pi"
    beta = str(np.array(data_frame["beta"])[0]/np.pi)[:4] + "pi"
    x = data_frame[target]
    ys = op(data_frame[cols_eig])
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(x, ys)
    if len(preds) != 0:
        ax.plot(preds["y"], ys)
        nombre = "pred_lineal/"
    #fin if 
    ax.set_xlabel(target)
    ax.set_ylabel(nombre_feat)
    ax.set_title("eta=" +  eta + ", beta=" + beta)
    ax.legend(cols_eig + cols_preds)
    plt.savefig("pictures/" + nombre + "eta=" +  eta + ",beta=" + beta + "_" + ".png")
    plt.close()
#fin for 

In [9]:
def graficar_todas(d_frame, n_eig, target, op = lambda x: x, preds = {}, nombre = "lineal/", nombre_feat = "l_moño"):
    for n in range(N_partes):
        start = N_phi_K*n
        end = N_phi_K*(n+1)
        ys = preds["y"][start:end] if len(preds) != 0 else []
        preds_fin = {"y": ys, "X": preds["X"].iloc[start:end]} if len(preds) != 0 else {}
        graficar_lambdas(d_frame.iloc[start:end], target, n_eig, preds_fin, op, nombre, nombre_feat)
    #fin for 
#fin función

In [10]:
n_comp = 3
target = "phi_K"
datos_copy = datos_combi.copy()
features_eig = list(map(lambda x: "eig_" + str(x+1), range(N_eig)))
features_dim = ["eta", "beta"]
features = features_dim + features_eig
pca_obj = PCA(n_components = n_comp)
pca_obj.set_output(transform="pandas")
X_eigs = datos_copy[features_eig]
pca_obj.fit(X_eigs)
X_pcas = pca_obj.transform(X_eigs)
X_dims = datos_copy[features_dim]
X_train = pd.concat((X_dims, X_pcas), axis = 1)
y_train = datos_copy[target]


In [11]:
datos_full = pd.concat((X_train, y_train), axis=1)
X_train.head()

,eta,beta,pca0,pca1,pca2
0,0.19635,1.047198,-42.909317,0.487427,-0.757292
1,0.19635,1.047198,-42.915680,0.485792,-0.755665
2,0.19635,1.047198,-42.922039,0.484162,-0.754045
3,0.19635,1.047198,-42.928394,0.482537,-0.752430
4,0.19635,1.047198,-42.934745,0.480917,-0.750823


In [12]:
graficar_todas(datos_full, n_comp, target, nombre = "PCA/", nombre_feat="pcas")